# 🚀 Azure APIM + MCP 서버 워크샵

## 엔터프라이즈급 Model Context Protocol 서버 구축 실습

![Workshop Banner](images/architecture.png)

---

## 📋 워크샵 개요

이 워크샵에서는 **Azure API Management (APIM)**와 **Azure Functions**를 활용하여 엔터프라이즈급 **MCP (Model Context Protocol)** 서버를 구축합니다.

### 학습 목표

1. **MCP 프로토콜 이해** - Model Context Protocol의 기본 개념과 구조
2. **Azure Functions MCP 서버** - HTTP Trigger 기반 MCP 서버 구현
3. **APIM 게이트웨이 구성** - 인증, Rate Limiting, 변환 정책 적용
4. **REST API → MCP 변환** - 기존 REST API를 MCP 서버로 노출
5. **엔터프라이즈 보안** - OpenID Connect, JWT 검증, External Entra ID
6. **모니터링 & 운영** - Application Insights, APIM Diagnostics

---

## 🎯 워크샵 구조

| 섹션 | 주제 | 난이도 |
|------|------|--------|
| **Section 0** | 환경 설정 및 사전 준비 | ⭐ |
| **Section 1** | Azure Functions로 MCP 서버 개발 | ⭐⭐ |
| **Section 2** | APIM으로 엔터프라이즈 게이트웨이 구성 | ⭐⭐ |
| **Section 3** | 외부 REST API를 MCP 서버로 변환 | ⭐⭐⭐ |
| **Section 4** | OpenID Connect 기반 인증 | ⭐⭐⭐ |
| **Section 5** | External Entra ID 연계 (B2B) | ⭐⭐⭐⭐ |
| **Section 6** | 모니터링 및 최적화 | ⭐⭐ |
| **Section 7** | 리소스 정리 | ⭐ |

---

## 🔧 사전 요구사항

### 필수 권한
- ✅ Azure Subscription Contributor 권한
- ✅ Entra ID Application 등록 권한
- ✅ APIM, Functions, Managed Identity 생성 권한

### 필수 도구
- ✅ [Azure CLI](https://docs.microsoft.com/cli/azure/) (최신 버전)
- ✅ [Python 3.12+](https://www.python.org/)
- ✅ [VS Code](https://code.visualstudio.com/) + [Azure Functions Extension](https://marketplace.visualstudio.com/items?itemName=ms-azuretools.vscode-azurefunctions)
- ✅ [Postman](https://www.postman.com/) 또는 REST Client (선택)

### 생성할 Azure 리소스
- Azure API Management (Basicv2 SKU)
- Azure Functions App (Python 3.12)
- Storage Account
- Application Insights
- Entra ID Applications (API + Client)

---

## 🏗️ 아키텍처

```
┌─────────────────┐
│   MCP Client    │
│  (GitHub        │
│   Copilot /     │
│   AI Agent)     │
└────────┬────────┘
         │ HTTPS + Bearer Token
         │
         ▼
┌────────────────────────────────────────────┐
│      Azure API Management (Gateway)        │
│  ┌──────────────────────────────────────┐  │
│  │ 🔐 JWT Validation (OpenID Connect)  │  │
│  │ 🚦 Rate Limiting & Quota            │  │
│  │ 🔄 Request/Response Transform       │  │
│  │ 📊 Logging & Diagnostics            │  │
│  │ 🌐 CORS & Header Management         │  │
│  └──────────────────────────────────────┘  │
└────────┬───────────────────────────────────┘
         │
    ┌────┴────┐
    ▼         ▼
┌──────────┐  ┌─────────────────┐
│  Azure   │  │  External       │
│ Functions│  │  REST APIs      │
│   MCP    │  │  (Weather,      │
│  Server  │  │   GitHub,       │
│          │  │   Custom APIs)  │
└──────────┘  └─────────────────┘
    │
    ▼
┌──────────────────┐
│ Application      │
│ Insights         │
│ (Monitoring)     │
└──────────────────┘
```

---

## 📚 참고 자료

- [Model Context Protocol Specification](https://spec.modelcontextprotocol.io/)
- [Azure API Management 문서](https://learn.microsoft.com/azure/api-management/)
- [Azure Functions 문서](https://learn.microsoft.com/azure/azure-functions/)
- [Microsoft Entra ID 문서](https://learn.microsoft.com/entra/identity/)

---

**준비되셨나요? 시작하겠습니다! 🎉**

<a id='0'></a>
# 0️⃣ 환경 설정 및 초기화

이 섹션에서는 워크샵에 필요한 환경을 설정하고 Azure 리소스를 생성합니다.

In [ ]:
# 필수 라이브러리 임포트
import os
import json
import subprocess
from datetime import datetime
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

# 워크샵 설정
WORKSHOP_CONFIG = {
    "subscription_id": os.getenv("AZURE_SUBSCRIPTION_ID", ""),
    "tenant_id": os.getenv("AZURE_TENANT_ID", ""),
    "location": os.getenv("AZURE_LOCATION", "koreacentral"),
    "resource_group": os.getenv("RESOURCE_GROUP_NAME", "rg-mcp-lab"),
    "apim_name": os.getenv("APIM_NAME", f"apim-mcp-{datetime.now().strftime('%Y%m%d')}"),
    "function_app_name": os.getenv("FUNCTION_APP_NAME", f"func-mcp-{datetime.now().strftime('%Y%m%d')}"),
    "storage_account": os.getenv("STORAGE_ACCOUNT_NAME", f"stmcp{datetime.now().strftime('%Y%m%d')}"),
    "mcp_protocol_version": "2024-11-05"
}

print("✅ 워크샵 설정 로드 완료")
print(json.dumps(WORKSHOP_CONFIG, indent=2, ensure_ascii=False))

In [ ]:
# Azure CLI 로그인 확인
result = subprocess.run(["az", "account", "show"], capture_output=True, text=True)

if result.returncode != 0:
    print("⚠️ Azure CLI 로그인이 필요합니다. 다음 명령어를 실행하세요:")
    print("   az login")
else:
    account = json.loads(result.stdout)
    print(f"✅ 로그인 완료: {account['user']['name']}")
    print(f"   구독: {account['name']}")
    print(f"   구독 ID: {account['id']}")
    
    # 설정 파일에 구독 ID 저장
    WORKSHOP_CONFIG["subscription_id"] = account['id']
    WORKSHOP_CONFIG["tenant_id"] = account['tenantId']

## 0.1 Azure CLI 로그인 및 구독 설정

In [ ]:
# 리소스 그룹 생성
rg_name = WORKSHOP_CONFIG["resource_group"]
location = WORKSHOP_CONFIG["location"]

print(f"📦 리소스 그룹 생성: {rg_name}")
result = subprocess.run([
    "az", "group", "create",
    "--name", rg_name,
    "--location", location
], capture_output=True, text=True)

if result.returncode == 0:
    print(f"✅ 리소스 그룹 생성 완료: {rg_name}")
else:
    print(f"ℹ️ 리소스 그룹이 이미 존재하거나 생성 중 오류 발생")
    print(result.stderr)

<a id='1'></a>
# 1️⃣ Azure Functions로 MCP 서버 개발

이 섹션에서는 Azure Functions를 사용하여 MCP (Model Context Protocol) 서버를 구현하고 배포합니다.

## 학습 목표
- MCP 프로토콜의 기본 구조 이해
- HTTP Trigger Function으로 MCP 엔드포인트 구현
- 로컬 테스트 및 Azure 배포

## 1.1 MCP 서버 코드 검토

먼저 이미 구현된 MCP 서버 코드를 살펴보겠습니다.

In [ ]:
# MCP 서버 코드 읽기
mcp_server_path = "src/mcp-function/function_app.py"

with open(mcp_server_path, 'r', encoding='utf-8') as f:
    mcp_code = f.read()

print(f"📄 MCP 서버 코드: {mcp_server_path}")
print(f"   라인 수: {len(mcp_code.splitlines())}")
print(f"   크기: {len(mcp_code)} bytes")
print("\n주요 구성 요소:")
print("  ✅ MCP 서버 메타데이터")
print("  ✅ 3개의 도구 (echo, get_current_time, calculate)")
print("  ✅ 5개의 HTTP 엔드포인트")
print("  ✅ 에러 처리 및 로깅")
print("\n코드 미리보기 (처음 50줄):")
print("=" * 80)
print('\n'.join(mcp_code.splitlines()[:50]))
print("=" * 80)

## 1.2 Bicep으로 인프라 배포

Azure Functions와 APIM 인프라를 Bicep 템플릿으로 배포합니다.

In [ ]:
# Bicep 배포
print("🚀 Bicep 템플릿으로 인프라 배포 중...")
print("   이 작업은 10-15분 정도 소요됩니다.")
print()

deployment_name = f"mcp-deployment-{datetime.now().strftime('%Y%m%d-%H%M%S')}"

result = subprocess.run([
    "az", "deployment", "group", "create",
    "--resource-group", WORKSHOP_CONFIG["resource_group"],
    "--template-file", "bicep/main.bicep",
    "--name", deployment_name,
    "--parameters",
    f"prefix={WORKSHOP_CONFIG.get('prefix', 'mcp')}",
    f"environment=dev"
], capture_output=True, text=True)

if result.returncode == 0:
    print("✅ 배포 완료!")
    
    # 배포 출력 가져오기
    outputs_result = subprocess.run([
        "az", "deployment", "group", "show",
        "--resource-group", WORKSHOP_CONFIG["resource_group"],
        "--name", deployment_name,
        "--query", "properties.outputs",
        "-o", "json"
    ], capture_output=True, text=True)
    
    if outputs_result.returncode == 0:
        outputs = json.loads(outputs_result.stdout)
        print("\n📊 배포된 리소스:")
        for key, value in outputs.items():
            print(f"   {key}: {value.get('value', 'N/A')}")
        
        # 설정 업데이트
        WORKSHOP_CONFIG["apim_gateway_url"] = outputs.get("apimGatewayUrl", {}).get("value", "")
        WORKSHOP_CONFIG["function_app_url"] = outputs.get("functionAppUrl", {}).get("value", "")
        WORKSHOP_CONFIG["function_app_name"] = outputs.get("functionAppName", {}).get("value", "")
else:
    print(f"❌ 배포 실패: {result.stderr}")
    print("\n💡 수동 배포 방법:")
    print(f"   az deployment group create \\")
    print(f"     --resource-group {WORKSHOP_CONFIG['resource_group']} \\")
    print(f"     --template-file bicep/main.bicep")

## 1.3 Functions 코드 배포

Azure Functions에 MCP 서버 코드를 배포합니다.

In [ ]:
# Functions 배포 (Azure Functions Core Tools 필요)
function_app_name = WORKSHOP_CONFIG.get("function_app_name", "")

if function_app_name:
    print(f"📦 Functions App에 코드 배포: {function_app_name}")
    print("   참고: Azure Functions Core Tools (func CLI)가 설치되어 있어야 합니다.")
    print()
    
    # 수동 배포 안내
    print("💡 배포 명령어:")
    print(f"   cd src/mcp-function")
    print(f"   func azure functionapp publish {function_app_name}")
    print()
    print("또는 VS Code에서:")
    print("   1. Azure Functions 확장 설치")
    print("   2. src/mcp-function 폴더 우클릭")
    print("   3. 'Deploy to Function App...' 선택")
else:
    print("⚠️ Function App 이름이 설정되지 않았습니다.")
    print("   먼저 인프라를 배포하세요.")

## 1.4 MCP 서버 테스트

배포된 Functions MCP 서버를 테스트합니다.

In [ ]:
# MCP 서버 테스트
import requests

function_url = WORKSHOP_CONFIG.get("function_app_url", "")

if function_url:
    try:
        # Health Check
        print("🧪 테스트 1: Health Check")
        response = requests.get(f"{function_url}/api/mcp/health", timeout=10)
        if response.status_code == 200:
            print(f"   ✅ {response.json()}")
        else:
            print(f"   ❌ 상태 코드: {response.status_code}")
        
        # List Tools
        print("\n🧪 테스트 2: List Tools")
        response = requests.get(f"{function_url}/api/mcp/tools", timeout=10)
        if response.status_code == 200:
            tools = response.json().get("tools", [])
            print(f"   ✅ {len(tools)}개 도구 발견:")
            for tool in tools:
                print(f"      - {tool['name']}: {tool['description']}")
        
        # Call Tool: Echo
        print("\n🧪 테스트 3: Echo Tool")
        payload = {
            "name": "echo",
            "arguments": {"message": "Hello from Workshop!"}
        }
        response = requests.post(
            f"{function_url}/api/mcp/tools",
            json=payload,
            timeout=10
        )
        if response.status_code == 200:
            result = response.json()
            print(f"   ✅ {result['content'][0]['text']}")
        
        print("\n✅ 모든 테스트 통과!")
        
    except requests.exceptions.RequestException as e:
        print(f"❌ 테스트 실패: {e}")
        print("   Functions가 배포되었는지 확인하세요.")
else:
    print("⚠️ Function URL이 설정되지 않았습니다.")

<a id='2'></a>
# 2️⃣ APIM으로 엔터프라이즈 게이트웨이 구성

Azure API Management를 MCP 서버의 게이트웨이로 설정합니다.

## 학습 목표
- APIM API 생성 및 Backend 연결
- 정책 (Policies) 적용
- Rate Limiting 및 CORS 설정

## 2.1 APIM 기본 정책 적용

APIM에 기본 정책을 적용합니다 (CORS, Rate Limiting, 로깅).

In [ ]:
# APIM 정책 파일 읽기
with open("apim-policies/base-policy.xml", 'r', encoding='utf-8') as f:
    base_policy = f.read()

print("📋 APIM 기본 정책:")
print("=" * 80)
print(base_policy[:1000])  # 처음 1000자 출력
print("..." if len(base_policy) > 1000 else "")
print("=" * 80)
print()
print("💡 이 정책은 다음을 포함합니다:")
print("   ✅ CORS 설정")
print("   ✅ Rate Limiting (100 calls/분)")
print("   ✅ 요청/응답 로깅")
print("   ✅ MCP 프로토콜 헤더 추가")
print("   ✅ 에러 응답 표준화")
print()
print("🔧 적용 방법:")
print("   1. Azure Portal에서 APIM 열기")
print("   2. APIs > MCP API 선택")
print("   3. 'All operations' 선택")
print("   4. Inbound processing > Policy code editor")
print("   5. 위 XML 내용 붙여넣기")

## 2.2 APIM을 통한 MCP 서버 테스트

APIM Gateway를 통해 MCP 서버를 호출합니다.

In [ ]:
# APIM Gateway를 통한 테스트
apim_url = WORKSHOP_CONFIG.get("apim_gateway_url", "")

if apim_url:
    # Subscription Key 가져오기 필요
    print("🔑 APIM Subscription Key가 필요합니다.")
    print()
    print("Key 가져오는 방법:")
    print("   1. Azure Portal > APIM 리소스")
    print("   2. Subscriptions 메뉴")
    print("   3. 'Built-in all-access subscription' 선택")
    print("   4. '...' > Show/hide keys > Primary key 복사")
    print()
    
    # 수동 입력 또는 환경 변수에서 로드
    subscription_key = os.getenv("APIM_SUBSCRIPTION_KEY", "")
    
    if subscription_key:
        try:
            headers = {"Ocp-Apim-Subscription-Key": subscription_key}
            
            # Health Check through APIM
            print(f"🧪 APIM을 통한 Health Check: {apim_url}/mcp/health")
            response = requests.get(f"{apim_url}/mcp/health", headers=headers, timeout=10)
            print(f"   상태: {response.status_code}")
            if response.status_code == 200:
                print(f"   ✅ {response.json()}")
                print(f"   헤더: X-MCP-Protocol-Version = {response.headers.get('X-MCP-Protocol-Version', 'N/A')}")
            
        except Exception as e:
            print(f"❌ 오류: {e}")
    else:
        print("⚠️ Subscription Key를 환경 변수에 설정하세요:")
        print('   export APIM_SUBSCRIPTION_KEY="your-key-here"')
else:
    print("⚠️ APIM Gateway URL이 설정되지 않았습니다.")

<a id='3'></a>
# 3️⃣ 외부 REST API를 MCP 서버로 변환

기존 REST API를 APIM 정책을 통해 MCP 형식으로 변환하여 노출합니다.

## 학습 목표
- REST API를 APIM Backend로 등록
- 요청/응답 변환 정책 작성
- Weather API 예제 구현

## 3.1 요청 변환 정책 검토

MCP 요청을 REST API 요청으로 변환하는 정책을 살펴봅니다.

In [ ]:
# 요청 변환 정책
with open("apim-policies/transform-request-policy.xml", 'r', encoding='utf-8') as f:
    request_policy = f.read()

print("📝 요청 변환 정책 (MCP → REST):")
print("=" * 80)
print(request_policy)
print("=" * 80)
print()
print("💡 변환 과정:")
print("   1. MCP Tool Call 수신:")
print('      {"name": "get_weather", "arguments": {"city": "Seoul"}}')
print()
print("   2. city 파라미터 추출")
print()
print("   3. REST API 요청으로 변환:")
print("      GET /data/2.5/weather?q=Seoul&appid=xxx&units=metric")
print()
print("   4. HTTP Method 변경: POST → GET")
print("   5. Body 제거 (GET 요청이므로)")

## 3.2 응답 변환 정책 검토

REST API 응답을 MCP 형식으로 변환하는 정책을 살펴봅니다.

In [ ]:
# 응답 변환 정책
with open("apim-policies/transform-response-policy.xml", 'r', encoding='utf-8') as f:
    response_policy = f.read()

print("📝 응답 변환 정책 (REST → MCP):")
print("=" * 80)
print(response_policy)
print("=" * 80)
print()
print("💡 변환 과정:")
print("   1. Weather API 응답 수신:")
print('      {"name": "Seoul", "main": {"temp": 5}, "weather": [{"description": "Clear"}]}')
print()
print("   2. MCP 형식으로 변환:")
print('      {')
print('        "content": [{')
print('          "type": "text",')
print('          "text": "도시: Seoul\\n온도: 5°C\\n날씨: Clear"')
print('        }]')
print('      }')
print()
print("   3. Content-Type 설정: application/json")
print("   4. 에러 발생 시 MCP 에러 형식으로 변환")

<a id='4'></a>
# 4️⃣ OpenID Connect 기반 인증

Entra ID와 OpenID Connect를 사용하여 MCP 서버에 인증을 추가합니다.

## 학습 목표
- Entra ID 앱 등록
- JWT 토큰 검증 정책 적용
- OAuth 2.0 Client Credentials Flow

## 4.1 Entra ID 앱 등록

Azure Portal에서 Entra ID 앱을 등록합니다.

In [ ]:
# Entra ID 앱 등록 가이드
print("🔐 Entra ID 앱 등록 단계:")
print()
print("=" * 80)
print("1️⃣ API 앱 등록 (MCP API)")
print("=" * 80)
print("   a. Azure Portal > Entra ID > App registrations > New registration")
print("   b. Name: MCP-API")
print("   c. Supported account types: Single tenant")
print("   d. Redirect URI: (비워둠)")
print("   e. Register 클릭")
print()
print("   f. 등록 후:")
print("      - Application (client) ID 복사")
print("      - Directory (tenant) ID 복사")
print()
print("   g. Expose an API:")
print("      - Application ID URI: api://mcp-api")
print("      - Add a scope:")
print("        * Scope name: mcp.access")
print("        * Who can consent: Admins and users")
print("        * Admin consent display name: Access MCP API")
print("        * Admin consent description: Allow access to MCP API")
print()
print("=" * 80)
print("2️⃣ Client 앱 등록 (MCP Client)")
print("=" * 80)
print("   a. App registrations > New registration")
print("   b. Name: MCP-Client")
print("   c. Register 클릭")
print()
print("   d. Certificates & secrets:")
print("      - New client secret")
print("      - Description: MCP Client Secret")
print("      - Expires: 6 months")
print("      - Add 클릭")
print("      - Value 복사 (한 번만 표시됨!)")
print()
print("   e. API permissions:")
print("      - Add a permission > My APIs > MCP-API")
print("      - Delegated permissions: mcp.access")
print("      - Add permissions 클릭")
print("      - Grant admin consent for [Tenant]")
print()
print("=" * 80)
print()
print("📝 환경 변수에 저장:")
print('   export AZURE_TENANT_ID="your-tenant-id"')
print('   export ENTRA_API_APP_ID="api-app-id"')
print('   export ENTRA_CLIENT_APP_ID="client-app-id"')
print('   export ENTRA_CLIENT_SECRET="client-secret"')

## 4.2 JWT 검증 정책 적용

APIM에서 JWT 토큰을 검증하는 정책을 적용합니다.

In [ ]:
# JWT 검증 정책
with open("apim-policies/jwt-validation-policy.xml", 'r', encoding='utf-8') as f:
    jwt_policy = f.read()

tenant_id = WORKSHOP_CONFIG.get("tenant_id", "{{TENANT_ID}}")
api_app_id = os.getenv("ENTRA_API_APP_ID", "{{API_APP_ID}}")

# 템플릿 변수 치환
jwt_policy_applied = jwt_policy.replace("{{TENANT_ID}}", tenant_id)
jwt_policy_applied = jwt_policy_applied.replace("{{API_APP_ID}}", api_app_id)

print("🔐 JWT 검증 정책:")
print("=" * 80)
print(jwt_policy_applied[:1500])  # 처음 1500자 출력
print("..." if len(jwt_policy_applied) > 1500 else "")
print("=" * 80)
print()
print("💡 주요 검증 항목:")
print("   ✅ OpenID Configuration (Entra ID)")
print("   ✅ Audience (aud): api://mcp-api")
print(f"   ✅ Issuer (iss): https://login.microsoftonline.com/{tenant_id}/v2.0")
print("   ✅ Scope (scp): mcp.access or user_impersonation")
print("   ✅ 사용자 정보 추출 (oid, upn, email)")
print()
print("🔧 APIM에 적용:")
print("   1. Azure Portal > APIM > APIs > MCP API")
print("   2. All operations > Inbound processing")
print("   3. 위 정책 XML을 기존 정책 <inbound> 태그 내에 추가")

## 4.3 토큰 발급 및 테스트

OAuth 2.0 Client Credentials Flow로 토큰을 발급받아 APIM을 호출합니다.

In [ ]:
# 토큰 발급
tenant_id = WORKSHOP_CONFIG.get("tenant_id", "")
client_id = os.getenv("ENTRA_CLIENT_APP_ID", "")
client_secret = os.getenv("ENTRA_CLIENT_SECRET", "")
api_app_id = os.getenv("ENTRA_API_APP_ID", "")

if all([tenant_id, client_id, client_secret, api_app_id]):
    print("🔑 OAuth 2.0 토큰 발급 중...")
    
    token_url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"
    token_data = {
        "client_id": client_id,
        "client_secret": client_secret,
        "scope": f"api://{api_app_id}/.default",
        "grant_type": "client_credentials"
    }
    
    try:
        response = requests.post(token_url, data=token_data, timeout=10)
        response.raise_for_status()
        
        token_response = response.json()
        access_token = token_response.get("access_token", "")
        
        if access_token:
            print("✅ 토큰 발급 성공!")
            print(f"   Token Type: {token_response.get('token_type', 'N/A')}")
            print(f"   Expires In: {token_response.get('expires_in', 'N/A')} seconds")
            print(f"   Access Token (처음 50자): {access_token[:50]}...")
            
            # 토큰을 환경 변수에 저장
            os.environ["MCP_ACCESS_TOKEN"] = access_token
            
            # 토큰으로 APIM 호출 테스트
            apim_url = WORKSHOP_CONFIG.get("apim_gateway_url", "")
            if apim_url:
                print(f"\n🧪 JWT 인증 테스트: {apim_url}/mcp/health")
                headers = {
                    "Authorization": f"Bearer {access_token}",
                    "Ocp-Apim-Subscription-Key": os.getenv("APIM_SUBSCRIPTION_KEY", "")
                }
                
                test_response = requests.get(f"{apim_url}/mcp/health", headers=headers, timeout=10)
                print(f"   상태: {test_response.status_code}")
                
                if test_response.status_code == 200:
                    print(f"   ✅ 인증 성공: {test_response.json()}")
                elif test_response.status_code == 401:
                    print("   ❌ 인증 실패 (401 Unauthorized)")
                    print("   JWT 검증 정책이 APIM에 적용되었는지 확인하세요.")
        else:
            print("❌ 토큰을 받지 못했습니다.")
            
    except requests.exceptions.RequestException as e:
        print(f"❌ 토큰 발급 실패: {e}")
        
else:
    print("⚠️ Entra ID 설정이 완료되지 않았습니다.")
    print("   다음 환경 변수를 설정하세요:")
    print("   - ENTRA_CLIENT_APP_ID")
    print("   - ENTRA_CLIENT_SECRET")
    print("   - ENTRA_API_APP_ID")
    print()
    print("💡 또는 src/client/get-token.sh 스크립트를 사용하세요.")

<a id='5'></a>
# 5️⃣ External Entra ID 연계 (고급)

외부 Entra ID 테넌트 사용자(B2B Guest)가 MCP 서버에 접근하는 시나리오를 구현합니다.

## 학습 목표
- Cross-tenant 접근 설정
- B2B Guest 사용자 초대
- External 사용자 토큰 검증

## 5.1 B2B Guest 사용자 초대

외부 테넌트의 사용자를 Guest로 초대합니다.

In [ ]:
# B2B Guest 사용자 초대 가이드
print("👥 External Entra ID 연계 시나리오:")
print()
print("=" * 80)
print("📌 시나리오 개요")
print("=" * 80)
print("  조직 A (contoso.com): APIM + MCP 서버 호스팅")
print("  조직 B (fabrikam.com): 파트너 조직")
print("  목표: fabrikam.com 사용자가 contoso.com의 MCP 서버 접근")
print()
print("=" * 80)
print("1️⃣ Cross-tenant Access 설정 (조직 A)")
print("=" * 80)
print("  a. Azure Portal > Entra ID > External Identities")
print("  b. Cross-tenant access settings")
print("  c. Organizational settings > Add organization")
print("  d. Tenant ID 또는 Domain 입력 (fabrikam.com)")
print("  e. Inbound access:")
print("     - B2B collaboration: ✅ Allow")
print("     - Applications: MCP-API 선택")
print()
print("=" * 80)
print("2️⃣ Guest 사용자 초대")
print("=" * 80)
print("  a. Entra ID > Users > New user > Invite external user")
print("  b. Email: user@fabrikam.com")
print("  c. Display name: Fabrikam User")
print("  d. Invite 클릭")
print("  e. 초대 이메일 발송됨")
print()
print("=" * 80)
print("3️⃣ 앱 권한 부여")
print("=" * 80)
print("  a. Entra ID > Enterprise applications > MCP-API")
print("  b. Users and groups > Add user/group")
print("  c. Guest 사용자 선택 (user_fabrikam.com#EXT#)")
print("  d. Assign 클릭")
print()
print("=" * 80)
print("4️⃣ JWT 정책 수정 (APIM)")
print("=" * 80)
print("  validate-jwt 정책의 <issuers> 태그에 추가:")
print('  <issuer>https://sts.windows.net/{fabrikam-tenant-id}/</issuer>')
print()
print("  또는 tid (Tenant ID) 클레임으로 검증:")
print('  <claim name="tid" match="any">')
print(f'    <value>{WORKSHOP_CONFIG.get("tenant_id", "your-tenant-id")}</value>')
print('    <value>{external-tenant-id}</value>')
print('  </claim>')
print()
print("=" * 80)
print("5️⃣ External 사용자 토큰 테스트")
print("=" * 80)
print("  a. External 사용자로 로그인")
print("  b. 토큰 발급 (동일한 Client Credentials Flow)")
print("  c. APIM 호출")
print("  d. JWT에서 tid 클레임 확인 (외부 테넌트 ID)")
print()
print("💡 보안 고려사항:")
print("  - MFA 필수 설정 (조건부 액세스)")
print("  - IP 제한")
print("  - 세션 타임아웃")
print("  - 감사 로그 모니터링")

<a id='6'></a>
# 6️⃣ 모니터링 및 최적화

Application Insights와 APIM Diagnostics를 활용하여 MCP 서버를 모니터링하고 최적화합니다.

## 학습 목표
- Application Insights 쿼리
- APIM 사용량 분석
- 성능 최적화

## 6.1 Application Insights 쿼리

Kusto 쿼리를 사용하여 로그를 분석합니다.

In [ ]:
# Application Insights 쿼리 예제
print("📊 유용한 Application Insights 쿼리:")
print()
print("=" * 80)
print("1️⃣ MCP Tool 호출 통계")
print("=" * 80)
print("""
requests
| where timestamp > ago(24h)
| where url contains "/mcp/tools"
| where success == true
| summarize Count = count() by name
| order by Count desc
""")
print()
print("=" * 80)
print("2️⃣ 응답 시간 분석")
print("=" * 80)
print("""
requests
| where timestamp > ago(24h)
| where url contains "/mcp/"
| summarize 
    avg(duration),
    percentile(duration, 50),
    percentile(duration, 95),
    percentile(duration, 99)
    by operation_Name
""")
print()
print("=" * 80)
print("3️⃣ 에러율 분석")
print("=" * 80)
print("""
requests
| where timestamp > ago(24h)
| where url contains "/mcp/"
| summarize 
    Total = count(),
    Errors = countif(success == false),
    ErrorRate = (countif(success == false) * 100.0) / count()
    by bin(timestamp, 1h)
| render timechart
""")
print()
print("=" * 80)
print("4️⃣ 사용자별 사용량")
print("=" * 80)
print("""
requests
| where timestamp > ago(7d)
| where url contains "/mcp/"
| extend userId = tostring(customDimensions["X-User-Id"])
| where isnotempty(userId) and userId != "anonymous"
| summarize 
    CallCount = count(),
    UniqueTools = dcount(operation_Name)
    by userId
| order by CallCount desc
""")
print()
print("=" * 80)
print("5️⃣ Rate Limit 적중 분석")
print("=" * 80)
print("""
traces
| where timestamp > ago(24h)
| where message contains "rate limit"
| summarize Count = count() by bin(timestamp, 1h), client_IP
| render timechart
""")
print()
print("🔗 Application Insights 접근:")
print("   Azure Portal > Application Insights 리소스 > Logs")

---
<a id='6.2'></a>
## 6.2 APIM 진단 및 성능 모니터링

APIM의 Analytics와 진단 설정을 확인합니다.

In [ ]:
# APIM 진단 설정 확인
print("🔍 APIM 진단 설정 확인:")
print()

# 1. APIM Analytics 확인
print("1️⃣ APIM Analytics 데이터:")
print("""
Azure Portal > API Management 서비스 > Analytics 탭에서:
  - API 호출 통계
  - 응답 시간 분포
  - 지리적 분포
  - 가장 많이 호출된 API
  - 에러율 추이
""")
print()

# 2. 진단 로그 설정
print("2️⃣ 진단 로그 설정:")
print("""
Azure Portal > API Management > Diagnostic settings > Add diagnostic setting

체크할 항목:
  ✅ GatewayLogs - 모든 API 요청/응답 로그
  ✅ WebSocketConnectionLogs - WebSocket 연결 로그
  ✅ AllMetrics - 성능 메트릭

전송 대상:
  ✅ Send to Log Analytics workspace
  ✅ Archive to storage account (선택사항)
""")
print()

# 3. APIM 로그 쿼리
print("3️⃣ APIM Gateway Logs 쿼리 (Log Analytics):")
print("""
ApiManagementGatewayLogs
| where TimeGenerated > ago(24h)
| where ApiId contains "mcp"
| summarize 
    TotalCalls = count(),
    AvgDuration = avg(TotalTime),
    ErrorCount = countif(ResponseCode >= 400)
    by bin(TimeGenerated, 1h), OperationId
| render timechart
""")
print()

# 4. 성능 메트릭 쿼리
print("4️⃣ 성능 메트릭 쿼리:")
print("""
AzureMetrics
| where ResourceProvider == "MICROSOFT.APIMANAGEMENT"
| where TimeGenerated > ago(24h)
| where MetricName in ("Requests", "Duration", "Capacity")
| summarize avg(Average) by MetricName, bin(TimeGenerated, 1h)
| render timechart
""")
print()

print("✅ APIM 모니터링 설정 완료!")
print("💡 Tip: Azure Monitor Alerts를 설정하여 임계값 초과 시 알림을 받을 수 있습니다.")

---
<a id='6.3'></a>
## 6.3 성능 최적화 팁

MCP + APIM 아키텍처의 성능을 최적화하는 방법입니다.

In [ ]:
# 성능 최적화 가이드
print("⚡ MCP + APIM 성능 최적화 체크리스트:")
print()

print("=" * 80)
print("1️⃣ Azure Functions 최적화")
print("=" * 80)
print("""
✅ 적절한 Hosting Plan 선택:
   - Consumption Plan: 간헐적 사용, 낮은 비용 (현재 설정)
   - Premium Plan: 예측 가능한 성능, VNet 통합
   - Dedicated Plan: 지속적인 워크로드

✅ 함수 최적화:
   - 불필요한 모듈 import 제거
   - 전역 변수로 재사용 가능한 객체 캐싱
   - 비동기 처리 활용 (async/await)
   - 메모리 사용량 모니터링

✅ Cold Start 최소화:
   - Premium Plan 사용 (Always Ready instances)
   - 함수 워밍업 스케줄 설정
   - 의존성 최소화
""")
print()

print("=" * 80)
print("2️⃣ APIM 최적화")
print("=" * 80)
print("""
✅ Caching 정책 활용:
   <cache-lookup vary-by-developer="false" vary-by-developer-groups="false" />
   <cache-store duration="3600" />

✅ Backend 연결 최적화:
   - Connection pooling 활성화
   - Timeout 적절히 설정
   - Retry 정책 구성

✅ Rate Limiting 세밀화:
   - 사용자별/구독별 제한 설정
   - Burst 허용량 조정
   - Quota 정책 활용

✅ 응답 압축:
   - gzip 압축 활성화
   - 큰 페이로드 최적화
""")
print()

print("=" * 80)
print("3️⃣ MCP 프로토콜 최적화")
print("=" * 80)
print("""
✅ Tool 설계:
   - Tool 실행 시간 최소화
   - 무거운 작업은 비동기 처리
   - 결과 크기 제한 (페이징 고려)

✅ 에러 핸들링:
   - 명확한 에러 메시지 제공
   - Retry 가능한 에러 구분
   - Circuit breaker 패턴 고려

✅ 보안과 성능 균형:
   - JWT 캐싱 (토큰 재사용)
   - 불필요한 검증 최소화
   - JWKS 캐싱 활용
""")
print()

print("=" * 80)
print("4️⃣ 모니터링 및 알림")
print("=" * 80)
print("""
✅ 주요 메트릭 추적:
   - 응답 시간 (p50, p95, p99)
   - 에러율 (4xx, 5xx)
   - 처리량 (requests/sec)
   - 리소스 사용률 (CPU, Memory)

✅ Alert 설정:
   - 응답 시간 > 1초
   - 에러율 > 5%
   - Rate limit 적중 빈도
   - 함수 실행 실패

✅ 대시보드 구성:
   - Azure Dashboard에 주요 메트릭 시각화
   - Grafana/Power BI 연동 고려
""")
print()

print("✅ 성능 최적화 가이드 완료!")
print("💡 Tip: 주기적으로 성능 테스트를 실행하고 병목 구간을 식별하세요.")

---
<a id='7'></a>
# 7. 리소스 정리 (Cleanup)

실습이 끝나면 생성된 Azure 리소스를 정리하여 불필요한 비용을 방지합니다.

---
<a id='7.1'></a>
## 7.1 리소스 그룹 삭제

⚠️ **주의**: 이 명령은 리소스 그룹과 그 안의 모든 리소스를 **영구적으로** 삭제합니다.

In [ ]:
# 리소스 그룹 삭제
import subprocess
import os
from dotenv import load_dotenv

load_dotenv()

RESOURCE_GROUP = os.getenv("RESOURCE_GROUP")

print(f"⚠️  리소스 그룹 삭제: {RESOURCE_GROUP}")
print()
print("삭제될 리소스:")
print("  - API Management 서비스")
print("  - Azure Functions App")
print("  - Application Insights")
print("  - Storage Account")
print("  - Log Analytics Workspace")
print("  - 모든 연결된 리소스")
print()

# 확인 메시지
confirmation = input(f"정말로 '{RESOURCE_GROUP}' 리소스 그룹을 삭제하시겠습니까? (yes/no): ")

if confirmation.lower() == 'yes':
    print()
    print("🗑️  리소스 그룹 삭제 중... (이 작업은 몇 분 소요될 수 있습니다)")
    
    try:
        result = subprocess.run(
            [
                "az", "group", "delete",
                "--name", RESOURCE_GROUP,
                "--yes",
                "--no-wait"
            ],
            capture_output=True,
            text=True,
            check=True
        )
        
        print("✅ 리소스 그룹 삭제 요청이 제출되었습니다.")
        print()
        print("삭제 상태 확인:")
        print(f"   az group show --name {RESOURCE_GROUP}")
        print()
        print("또는 Azure Portal에서 확인:")
        print(f"   https://portal.azure.com/#view/HubsExtension/BrowseResourceGroups")
        
    except subprocess.CalledProcessError as e:
        print(f"❌ 삭제 실패: {e.stderr}")
        print()
        print("수동으로 삭제하려면:")
        print(f"   az group delete --name {RESOURCE_GROUP} --yes")
else:
    print()
    print("❌ 삭제가 취소되었습니다.")
    print()
    print("💡 Tip: 리소스를 유지하려면 이 셀을 실행하지 마세요.")

---
<a id='7.2'></a>
## 7.2 개별 리소스 정리 (선택사항)

리소스 그룹을 유지하면서 특정 리소스만 삭제하고 싶은 경우 사용합니다.

In [ ]:
# 개별 리소스 삭제 명령어
import os
from dotenv import load_dotenv

load_dotenv()

RESOURCE_GROUP = os.getenv("RESOURCE_GROUP")
APIM_NAME = os.getenv("APIM_SERVICE_NAME")
FUNCTION_APP_NAME = os.getenv("FUNCTION_APP_NAME")

print("🗑️  개별 리소스 삭제 명령어:")
print()

print("=" * 80)
print("1️⃣ API Management 서비스 삭제 (가장 비용이 많이 드는 리소스)")
print("=" * 80)
print(f"az apim delete \\")
print(f"  --name {APIM_NAME} \\")
print(f"  --resource-group {RESOURCE_GROUP} \\")
print(f"  --yes --no-wait")
print()
print("💰 비용 절감: ~$50-100/month (Basicv2 SKU 기준)")
print()

print("=" * 80)
print("2️⃣ Functions App 삭제")
print("=" * 80)
print(f"az functionapp delete \\")
print(f"  --name {FUNCTION_APP_NAME} \\")
print(f"  --resource-group {RESOURCE_GROUP}")
print()

print("=" * 80)
print("3️⃣ Storage Account 삭제")
print("=" * 80)
print(f"az storage account delete \\")
print(f"  --name {FUNCTION_APP_NAME}storage \\")
print(f"  --resource-group {RESOURCE_GROUP} \\")
print(f"  --yes")
print()

print("=" * 80)
print("4️⃣ Application Insights 삭제")
print("=" * 80)
print(f"az monitor app-insights component delete \\")
print(f"  --app {FUNCTION_APP_NAME}-insights \\")
print(f"  --resource-group {RESOURCE_GROUP}")
print()

print("=" * 80)
print("5️⃣ Log Analytics Workspace 삭제")
print("=" * 80)
print(f"az monitor log-analytics workspace delete \\")
print(f"  --workspace-name {FUNCTION_APP_NAME}-workspace \\")
print(f"  --resource-group {RESOURCE_GROUP} \\")
print(f"  --yes")
print()

print("✅ 개별 리소스 삭제 명령어 출력 완료")
print()
print("💡 Tip: 위의 명령어를 복사하여 터미널에서 실행하세요.")
print("⚠️  주의: APIM 삭제는 복구 불가능하며, 재생성 시 시간이 오래 걸립니다 (30-40분).")

---
<a id='7.3'></a>
## 7.3 비용 관리 팁

Azure 리소스를 유지하면서 비용을 최소화하는 방법입니다.

In [ ]:
# 비용 관리 팁
print("💰 Azure 비용 절감 가이드:")
print()

print("=" * 80)
print("1️⃣ API Management 비용 최적화")
print("=" * 80)
print("""
현재 SKU: Basicv2 (~$50-100/month)

비용 절감 옵션:
  ⭐ Developer SKU로 변경: ~$50/month
     - 개발/테스트 환경에 적합
     - SLA 없음, 스케일링 제한
     - 명령어: az apim update --name <NAME> --sku-name Developer

  ⭐ Consumption 플랜 고려:
     - 사용량 기반 과금 (첫 100만 건 무료)
     - Cold start 발생 가능
     - 낮은 트래픽 환경에 적합

  ⚠️  불필요한 APIM 기능 비활성화:
     - 사용하지 않는 API 제거
     - 불필요한 정책 최소화
""")
print()

print("=" * 80)
print("2️⃣ Azure Functions 비용 최적화")
print("=" * 80)
print("""
현재 플랜: Consumption (사용량 기반 과금)

비용 절감 팁:
  ✅ 실행 시간 최소화:
     - 불필요한 코드 제거
     - 효율적인 알고리즘 사용
     - 외부 API 호출 캐싱

  ✅ 메모리 할당 최적화:
     - 필요한 만큼만 메모리 사용
     - 불필요한 라이브러리 제거

  ✅ 무료 할당량 활용:
     - 월 100만 실행 무료
     - 400,000 GB-s 무료
""")
print()

print("=" * 80)
print("3️⃣ Storage & Monitoring 비용 최적화")
print("=" * 80)
print("""
비용 절감 팁:
  ✅ Log 보관 정책 설정:
     - Application Insights: 90일 → 30일
     - Storage Account: 오래된 로그 자동 삭제
     - Log Analytics: 샘플링 비율 조정

  ✅ 불필요한 진단 로그 비활성화:
     - 개발 완료 후 상세 로깅 수준 낮추기
     - Verbose 로그 → Warning 이상만

  ✅ Storage Account 타입 최적화:
     - Hot → Cool 티어 (로그용)
     - LRS 복제 전략 (개발 환경)
""")
print()

print("=" * 80)
print("4️⃣ 비용 모니터링 설정")
print("=" * 80)
print("""
✅ Cost Management 활용:
   Azure Portal > Cost Management + Billing > Cost analysis

✅ 예산 경고 설정:
   az consumption budget create \\
     --budget-name "monthly-budget" \\
     --amount 100 \\
     --time-grain Monthly \\
     --resource-group <RESOURCE_GROUP>

✅ 비용 추적:
   - 일일 비용 확인
   - 예상치 못한 증가 알림 설정
   - 리소스별 비용 분석
""")
print()

print("=" * 80)
print("5️⃣ 개발 환경 자동화")
print("=" * 80)
print("""
✅ 스케줄 기반 시작/중지:
   - 업무 시간에만 APIM 실행
   - Azure Automation 활용
   - 야간/주말 자동 중지

✅ 환경 분리:
   - 개발 환경: Developer SKU + Consumption
   - 운영 환경: Standard SKU + Premium Functions
""")
print()

print("✅ 비용 관리 가이드 완료!")
print()
print("📊 현재 예상 비용:")
print("   - APIM Basicv2: ~$60/month")
print("   - Functions Consumption: ~$5/month (낮은 사용량 가정)")
print("   - Storage + Monitoring: ~$10/month")
print("   - 총 예상: ~$75/month")
print()
print("💡 Tip: Azure Cost Calculator로 정확한 비용 예측 가능")
print("   https://azure.microsoft.com/pricing/calculator/")

---

# 🎉 워크샵 완료!

축하합니다! Azure API Management와 Azure Functions를 활용한 MCP 서버 구축 워크샵을 완료하셨습니다.

## 📚 학습한 내용

이 워크샵에서 다음 내용을 학습하고 실습했습니다:

### 1️⃣ Azure Functions MCP 서버 구축
- Model Context Protocol (MCP) 2024-11-05 표준 구현
- Python 3.12 기반 HTTP Trigger 함수 개발
- 3가지 MCP Tool 구현 (echo, get_current_time, calculate)
- Health check 및 정보 조회 엔드포인트

### 2️⃣ Azure API Management 게이트웨이 설정
- APIM Basicv2 SKU를 사용한 엔터프라이즈 게이트웨이 구성
- CORS, Rate Limiting, Backend 연결 정책 설정
- API 정의 및 운영 관리

### 3️⃣ REST API ↔ MCP 변환
- 클라이언트 REST 요청을 MCP JSON-RPC로 변환
- MCP 응답을 RESTful 형식으로 변환
- 헤더 전달 및 컨텍스트 유지

### 4️⃣ OpenID Connect 인증 통합
- Entra ID (Azure AD) App Registration 구성
- OAuth 2.0 Client Credentials Flow 구현
- JWT 토큰 검증 정책 (validate-jwt)
- Bearer 토큰 기반 보안 적용

### 5️⃣ 외부 Entra ID 연동
- B2B Guest User 초대 및 관리
- 멀티 테넌트 인증 시나리오
- 외부 조직과의 안전한 API 공유

### 6️⃣ 모니터링 및 최적화
- Application Insights 쿼리 (Kusto)
- APIM Analytics 및 진단 로그
- 성능 최적화 체크리스트
- 비용 관리 전략

### 7️⃣ 인프라스트럭처 as Code
- Bicep 템플릿을 사용한 자동화된 배포
- Azure CLI를 활용한 리소스 관리
- 환경 변수 기반 구성

---

## 🚀 다음 단계

워크샵을 완료한 후 다음 활동을 고려해보세요:

### 🔧 프로젝트 확장
- **추가 MCP Tools 개발**: 실제 비즈니스 로직 구현 (데이터베이스 조회, 외부 API 호출 등)
- **WebSocket 지원**: 실시간 스트리밍을 위한 MCP over WebSocket 구현
- **AI 에이전트 통합**: OpenAI Assistant API나 Azure OpenAI와 MCP 연동

### 🔐 보안 강화
- **Managed Identity**: APIM ↔ Functions 간 Managed Identity 인증 적용
- **API Key Rotation**: 자동 키 로테이션 메커니즘 구현
- **IP Filtering**: APIM에서 허용된 IP만 접근 가능하도록 설정

### 📊 운영 개선
- **CI/CD 파이프라인**: GitHub Actions/Azure DevOps를 사용한 자동 배포
- **Multi-Region 배포**: 글로벌 사용자를 위한 다중 지역 배포
- **Auto-scaling**: Premium Functions Plan으로 업그레이드하여 자동 확장

### 🧪 고급 시나리오
- **GraphQL Gateway**: APIM에서 GraphQL 엔드포인트 노출
- **Event-driven Architecture**: Event Grid/Service Bus와 통합
- **Microservices Mesh**: 여러 MCP 서버를 오케스트레이션

---

## 📖 참고 자료

### MCP (Model Context Protocol)
- [MCP 공식 문서](https://modelcontextprotocol.io/)
- [MCP Specification](https://spec.modelcontextprotocol.io/)
- [MCP GitHub Repository](https://github.com/modelcontextprotocol)

### Azure API Management
- [APIM 공식 문서](https://learn.microsoft.com/azure/api-management/)
- [APIM 정책 참조](https://learn.microsoft.com/azure/api-management/api-management-policies)
- [APIM 샘플 정책](https://github.com/Azure/api-management-policy-snippets)

### Azure Functions
- [Functions 공식 문서](https://learn.microsoft.com/azure/azure-functions/)
- [Python Functions 개발자 가이드](https://learn.microsoft.com/azure/azure-functions/functions-reference-python)
- [Functions Best Practices](https://learn.microsoft.com/azure/azure-functions/functions-best-practices)

### Entra ID (Azure AD)
- [Entra ID 문서](https://learn.microsoft.com/entra/identity/)
- [OAuth 2.0 및 OpenID Connect](https://learn.microsoft.com/entra/identity-platform/v2-protocols)
- [B2B Collaboration](https://learn.microsoft.com/entra/external-id/what-is-b2b)

---

## 💬 피드백 및 기여

이 워크샵에 대한 피드백이나 개선 제안이 있으시면 언제든지 공유해주세요!

- GitHub Issues를 통한 버그 리포트
- Pull Request를 통한 코드 개선
- 커뮤니티 포럼에서 질문 및 토론

---

## 🙏 감사합니다!

이 워크샵을 완료해주셔서 감사합니다. Azure와 MCP를 활용한 엔터프라이즈 통합에 대해 유용한 지식을 얻으셨기를 바랍니다.

행운을 빕니다! 🚀

---

**마지막 업데이트**: 2024
**워크샵 버전**: 1.0
**제작**: Azure API Management + MCP Workshop